# Neural Network for Miles per Gallon
Using a dataset from [Heaton Research](https://data.heatonresearch.com/data/t81-558/auto-mpg.csv), we extract information on

- cylinders
- displacement
- horsepower
- weight
- acceleration
- year

in order to predict the miles-per-gallon (mpg). We will use a sequential neural network (NN) to achieve this.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import metrics

### Fetch the data
We can use pandas to read a csv file over HTTP:

In [2]:
# fetch the data
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?']
)

In [3]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


### Feature extraction
Next, we assemble the data in a form we can feed to the NN. We will fill missing inputs with the **median** for that column.

In [4]:
cars = df['name']

# handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# pandas to numpy
columns = ['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']
x = df[columns].values
y = df['mpg'].values # regression

### Build the NN
We use two layers, with a single output neuron. The complexity of network size and number of layers is a hyperparameter problem, and there exists much debate as to how to make an appropriate choice. Too complex of a network can over-fit to the data, and too simple of a model may be highly error-prone.

The rectifier linear unit ([ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))) is the common
$$
f(x) = max(0, x)
$$
to avoid gradient roll-off in the sigmoid or hyperbolic functions.

In [5]:
# build the neural network

model = tf.keras.models.Sequential()

model.add( # hidden layer 1
    tf.keras.layers.Dense(
        25, 
        input_dim=x.shape[1], 
        activation='relu'
    )
)
model.add( # hidden layer 2
    tf.keras.layers.Dense(
        10, activation='relu' # rectifier linear unit
    )
)
model.add( # output layer
    tf.keras.layers.Dense(1)
)
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=0,epochs=100)

Next we evaluate our model as RMS error deviation
$$
RMSE(\hat{\theta}) = \sqrt{E((\hat{\theta} - \theta)^2)}
$$
with the error function normalizing over the data set.

In [6]:
prediction = model.predict(x)
score = np.sqrt(metrics.mean_squared_error(prediction, y))
print(f"Final rms score {score}")

Final rms score 10.205322898894943
